<center><h1>Graph project: Epidemic spreading</h1></center>

<center><h4>ENSEEIHT SN</h4></center>

<div style="width:75%;margin:0 auto;">
    
## Introduction
<a id='intro'></a>
    
<p style="text-align:justify;"> As you know, a new epidemic has overwhelmed the world, COVID-19 jeopardizes people and changes our habits. It is easy to realise that knowing how illnesses spread is vital to our own protection. How can we predict whether a disease will cause an epidemic, how many people it will infect, which people it will infect, and whether or not it is dangerous to society as a whole ? Also, how can we determine which techniques to use in fighting an epidemic once it begins ? One way to answer all of these questions is through <strong>mathematical modeling</strong>. </p>

<p style="text-align:justify;"> In this work you will have to review different epidemic modelings relying all on the representation by graphs of a human network called a <strong>contact network</strong>. A vertex in a contact network represents an individual and an edge between two vertices represents a contact between two individuals. The disease only spread from individual to individual if they are in contact, so through the edges. This representation is actually really common in research, and a lot of state-of-the-art modeling are built over it. From these different models you will be asked to draw conclusions from experiments on varying contact networks</p>

<p style="text-align:justify;"> For readability and ease of use, this project will be carried on a Jupyter Notebook, hence code and question answering have to be written in this unique file. This is a <strong>DUO</strong> project, no group of one person will be accepted, the duo has to be composed of same TD group students, if the number of students in the TD group is odd we will accept one group composed of three students. It will be coded in Julia using the LightGraphs package. <strong>BEWARE:</strong> 
    
* If the code does not provide good results, its readability as well as its comments are essential for the corrector to potentially find some notation points.
* The specifications of the functions have to be strictly respected.
* Do not neglect written questions they stand for an important part of the notation, you are not only evaluated on the coding. Also, even so a written question may not ask you directly to code or provide results from code, support your arguments when possible with a runable example is very welcome and sometimes even expected.
* Any initiatives and additional efforts bringing contents and thoughts out of the question scope may result in bonus points if pertinent.
</p> 
    
<p style="text-align:justify;"> Deliverable: You will deliver your work on moodle before <strong style="font-size:1.3em">24.01.2021</strong> in a <strong style="font-size:1.3em">.tar</strong> containing the notebook with your codes and your written answers, and the different graph figures in .png you will generate. The corrector will use the student N7 computers for running your code, so take care of verifying that your work is running as expected on these computers !</p>

LightGraphs documentation: https://juliagraphs.org/LightGraphs.jl/v1.2/index.html
    
<!---
http://makie.juliaplots.org/stable/basic-tutorial.html#Adding-plots-to-a-Scene
TODO: afficher l'historique gplot des graphes durant la simu pour voir visuellement ce que ça donne. Peut etre necessiter de locker l'affichage graphique.
afficher pour tau fixer sur different graphe de degré le nombre max d'infecté durant la simu.
    
Faire coder le modèle SAIS et SIR

    Pour SIR faire la courbe dans un des sites dans les favoris
    
    Expliquer en quoi la distance et Jordan ne sont pas tout le temps optimal. Expliquer pourquoi le DMP peut présenter de meilleur résultat.

Demander d'imaginer d'autres modèle inspirer de maladie
--->

<div style="width:75%;margin:0 auto;">

## Environment and packages installation
<a id='env'></a>
    
<p style="text-align:justify;"> <strong>IMPORTANT</strong>: For evaluation, coding questions have to run with no additional packages ! Only the ones present here ! However if you want to use another package to go further in your answer and add bonus contents, take care of separating the cells and precising which packages you are using.</p>
    

In [1]:
# Download packages

import Pkg; 
Pkg.add("LightGraphs")
Pkg.add("GraphPlot")
Pkg.add("Colors")
Pkg.add("CairoMakie")
Pkg.add("StatsBase")
Pkg.add("Plots")
Pkg.add("JLD2")
Pkg.add("Compose")


   Updating registry at `C:\Users\Nitro\.julia\registries\General`
  Resolving package versions...
No Changes to `C:\Users\Nitro\.julia\environments\v1.5\Project.toml`
No Changes to `C:\Users\Nitro\.julia\environments\v1.5\Manifest.toml`
  Resolving package versions...
No Changes to `C:\Users\Nitro\.julia\environments\v1.5\Project.toml`
No Changes to `C:\Users\Nitro\.julia\environments\v1.5\Manifest.toml`
  Resolving package versions...
No Changes to `C:\Users\Nitro\.julia\environments\v1.5\Project.toml`
No Changes to `C:\Users\Nitro\.julia\environments\v1.5\Manifest.toml`
  Resolving package versions...
No Changes to `C:\Users\Nitro\.julia\environments\v1.5\Project.toml`
No Changes to `C:\Users\Nitro\.julia\environments\v1.5\Manifest.toml`
  Resolving package versions...
No Changes to `C:\Users\Nitro\.julia\environments\v1.5\Project.toml`
No Changes to `C:\Users\Nitro\.julia\environments\v1.5\Manifest.toml`
  Resolving package versions...
No Changes to `C:\Users\Nitro\.julia\environme

In [6]:
# Import packages
using LightGraphs
using GraphPlot
using Colors
using CairoMakie
using StatsBase
using Plots
using JLD2
using Compose

<div style="width:75%;margin:0 auto;">

## Part 1 - SIS model
<a id='part1'></a>

<p style="text-align:justify;">SIS is a compartmental model, ie a model where the population is divided into subgroups that represent the disease status of its members. SIS stands for Susceptible $\rightarrow$ Infected $\rightarrow$ Susceptible where the susceptible group contains those who remain susceptible to the infection, and the infected group consists of those who not only have the disease but are also in the contagious period of the disease.</p>
    
<p style="text-align:justify;"> Combine with a contact network approach, this model can capture contact patterns (family, company, friends). Each vertex represents an individual in the host population, and contacts between two individuals are represented by an edge that connects the two. The probability of transmitting the disease from an infected to a susceptible individual along one of these edges or contacts is $\beta$ (=<strong>infection rate</strong>). The probability to cure is $\alpha$ (= <strong>curing rate</strong>). </p>

    
<p style="text-align:justify;">In order for a disease to begin spreading through a network, the disease must be introduced into the population, either through infecting a proportion of the population or through infecting one individual. As time moves forward, the disease will spread away from those initially infected, and two things may occur simultaneously at each time step $t$. First, each infected individual will spread disease to each of its contacts with a probability $\beta$. Secondly, each infectious individual will recover at a rate, $\alpha$ , at which point the individual will then no longer infect any of its contacts. After the disease has run its course, we can determine how the disease affected the network by calculating various quantities that help us better understand the outbreak.
</p>

<br>
    
<p style="font-size:0.9em">P. Van Mieghem, J. Omic, R. E. Kooij, <em>“Virus Spread in Networks”</em>,
IEEE/ACM Transaction on Networking (2009)<p>


<!---
<div style="background-color:rgba(0, 0, 0, 0.0470588); padding:10px 0;font-family:monospace;">
<font size = 4.5px><b>Algorithm 1:</b> Girvan Newman</font><br>
    <div style="margin-left:50px;border-left:2px solid black;padding-left:10px;">
    <b> WHILE </b> there are still edges<b> OR </b> desired nb of communities unreached <b>DO</b>
        <div style="margin-left:50px;border-left:2px solid black;padding-left:10px;">
        Calculate betweeness of all edges <br>
        Remove the edge with the highest betweeness <br>
        Calculate the number of strongly connected components
        </div>
    <b> END WHILE </b>   
    </div>
</div>
--->

<div style="width:75%;margin:0 auto;">

### 1.1 Contact networks sample

In [24]:
"""karat7: A graph representing the karate club of N7 and the connections between the persons in this club.
There are 34 people in this network. It is actually inspired by one of the most famous problem in graph
theory: the Zachary's karate club.
"""
karat7 = smallgraph(:karate)
nodecolor = [colorant"lightseagreen"]
draw(PNG("part-1/blank_karat7.png", 100cm, 100cm), gplot(karat7,nodefillc=nodecolor))

In [25]:
"""n7_2A: A graph representing the second year students at N7. Each department (SN, MF2E, 3EA) form a 
community where connections are denser, connections between department are rarer.
"""
c=[[10,0,0] [0.1,10,0] [0.1,0.1,10]]
n=[100,70,50]
n7_2A = stochastic_block_model(c,n)
nodecolor = [colorant"lightseagreen"]
draw(PNG("part-1/blank_n7_2A.png", 100cm, 100cm), gplot(n7_2A,nodefillc=nodecolor))

In [26]:
"""toulouse_neigh: A graph representing a neighborhood composed of 1000 people in Toulouse.
"""
toulouse_neigh = barabasi_albert(1000, 1)
nodecolor = [colorant"lightseagreen"]
draw(PNG("part-1/blank_toulouse_neigh.png", 100cm, 100cm), gplot(toulouse_neigh,nodefillc=nodecolor))

<div style="width:75%;margin:0 auto;">
  
### 1.2 Introduce the infection
    
<p style="text-align:justify;">We denote by <code>state</code> a vector containing the disease status of each vertex where Susceptible=0 and Infected=1. Then <code>state</code> is an <code>Array{Int32,1}</code> of length the number of vertices. This array in addition of a graph (represented internally by an adjacency matrix or an adjacency list) will be the data structure of our model.</p>
    
<span style="font-size:0.9em">In <code>Array{Int32,1}</code>, <code>Int32</code> refers to the kind of data in the array, here 32 bits integers, <code>1</code> refers to the dimension of the array, here we have a 1-dimensional structure so a vector.</span>

<div style="width:75%;margin:0 auto;">

<strong style="color:cornflowerblue">Question 1 (code):</strong> For each graph in the graph sample (<code>karat7</code>, <code>n7_2A</code>, <code>toulouse_neigh</code>) initialize the state array by assigning each vertex to susceptible and add randomly one or numerous infected people. Save the graph as a .png image using <code>gplot</code> and <code>draw</code>, infected should appear in a different color (<code>colorant"orange"</code>).
    
Due to a bug on certain version of Jupyter Notebook, the graph <span style="font-size:1.3em">figures should be saved in a file and not plot inside the notebook </span> !!!
    
Gplot GitHub: https://github.com/JuliaGraphs/GraphPlot.jl
    
Gplot examples: https://juliagraphs.org/GraphPlot.jl/

In [67]:
# Infect Karat7
function reset_karat7(nbInf = 0)
    NV = nv(karat7)
    state_karat7 = zeros(Int32, NV)

    if nbInf == 0                 
        nbInf = Int(rand((1:NV/2))) # I thought it would be more interesting if 
                                    # the starting number of cases were under half of the population
    end
    for _ in 1:nbInf # Infecting nbInf persons
        randomId = Int(rand(1:NV))
        while state_karat7[randomId] == 1
            randomId = Int(rand(1:NV))
        end
        state_karat7[randomId] = 1
    end
    return state_karat7
end

NV = nv(karat7)
NE = ne(karat7)
state_karat7 = reset_karat7()

nodecolor = [colorant"lightseagreen", colorant"orange"]
nodefillc = nodecolor[state_karat7 + ones(Int32,NV)]

draw(PNG("part-1/Question_1_karat7.png", 100cm, 100cm), gplot(karat7,nodefillc=nodefillc))

![out](part-1/Question_1_karat7.png)

In [68]:
# Infect n7_2A



function reset_n7_2A(nbInf = 0)
    NV = nv(n7_2A)
    state_n7_2A = zeros(Int32, NV)
    
    if nbInf == 0                       
        nbInf = Int(rand((1:NV/2))) # I thought it would be more interesting if 
                                    # the starting number of cases were under half of the population
    end


    for _ in 1:nbInf # Infecting nbInf persons
        randomId = Int(rand(1:NV))
        while state_n7_2A[randomId] ==1
            randomId = Int(rand(1:NV))
        end
        state_n7_2A[randomId] = 1
    end
    
    return state_n7_2A
end

NV = nv(n7_2A)
state_n7_2A = reset_n7_2A()

nodecolor = [colorant"lightseagreen", colorant"orange"]
nodefillc = nodecolor[state_n7_2A + ones(Int32,NV)]

draw(PNG("part-1/Question_1_n7_2A.png", 100cm, 100cm), gplot(n7_2A,nodefillc=nodefillc))

![out](part-1/Question_1_n7_2A.png)

In [69]:
# Infect toulouse_neigh
function reset_toulouse_neigh(nbInf = 0)
    NV = nv(toulouse_neigh)
    state_toulouse_neigh = zeros(Int32, NV)

    if nbInf == 0                      
        nbInf = Int(rand((1:NV/2))) # I thought it would be more interesting if 
                                    # the starting number of cases were under half of the population
    end
    
    for _ in 1:nbInf # Infecting nbInf persons
        randomId = Int(rand(1:NV))
        while state_toulouse_neigh[randomId] ==1
            randomId = Int(rand(1:NV))
        end
        state_toulouse_neigh[randomId] = 1
    end

    return state_toulouse_neigh
end
NV = nv(toulouse_neigh)
NE = ne(toulouse_neigh)
state_toulouse_neigh = zeros(Int32, NV)
state_toulouse_neigh = reset_toulouse_neigh()


nodecolor = [colorant"lightseagreen", colorant"orange"]
nodefillc = nodecolor[state_toulouse_neigh + ones(Int32,NV)]

draw(PNG("part-1/Question_1_toulouse_neigh.png", 100cm, 100cm), gplot(toulouse_neigh,nodefillc=nodefillc))

![out](part-1/Question_1_toulouse_neigh.png)

<div style="width:75%;margin:0 auto;">

<strong style="color:cadetblue">Question 2 (written):</strong> What do you think/predict about the influence of the initial number of infected people and their locations on the evolution of an SIS model epidemic ?

<br>
    
<div style="background-color:#E7F1D1"><strong>Answer:
    I believe what matters the most is the ratio $\beta$ / $\alpha$, if it's greater than one, the number of cases should rise, and if it's less, the number of cases should go down and eventually reach zero. The starting number of cases should only affect the time the model takes to reach zero case / to infect all people
    
</strong></div> 

<div style="width:75%;margin:0 auto;">
  
### 1.3 Spread the infection

<div style="width:75%;margin:0 auto;">
      
<strong style="color:cornflowerblue">Question 3 (code):</strong> Implement the <code>function SIS</code> (respect the header and the specifications). You can use <code>rand</code> to translate the probabilities. Test your algorithm on <code>karat7</code>, <code>n7_2A</code>, and <code>toulouse_neigh</code> with arbitrary $\beta$, $\alpha$, and $t$.
    
<span style="font-size:0.9em">The corrector should be able to write <code>new_state = SIS(net,state,beta,alpha,t)</code> with your code.</span>

In [70]:
function SIS(net,state,beta,alpha,t)
    """Take a contact network at a certain state and apply t time steps
    of an SIS model.
    
    PARAMS
       net (LightGraph): graph representing the contact network
       state (Array{Int32,1}): disease status of each vertex
       beta (Float64): infection rate
       alpha (Float64): curing rate
       t (Int32): number of time step
    
    RETURNS
        (Array{Int32,1}): The new state of the contact network after t time steps.
    """     
    if t == 0 
        return state
    end
    
    stateTemp = zeros(Int32, nv(net))
    
    for i in 1:nv(net) # Using a temp variable to make sure that the change is step by step and changing the state of 
                       # a node won't affect the current step
        stateTemp[i] = state[i] 
    end
    
    for i in 1:nv(net) # Infect the nodes
        if state[i] == 1 # if the node is infected
            for ngb in neighbors(net,vertices(net)[i]) # for each of their neighbours
               if rand() <= beta # infect them with a probablility of beta
                    stateTemp[ngb] = 1
                end
            end
        end
    end
    
    for i in 1:nv(net) # Cure the nodes
        # I don't want to try alpha as the same time as beta because I don't want infected node
        # to be healed then infected again
        if state[i] == 1 # if the node is infected
            if rand() <= alpha
                stateTemp[i] = 0
            end
        end
    end
    
    return SIS(net,stateTemp,beta,alpha,t-1) # Because recursive is sexy
    
end


SIS (generic function with 1 method)

## <strong style="color:cornflowerblue"> Before running the folloing cells, please run the cell related to gifs in the annex in order to display the animated simulation. It is how we decided to run tests. </strong>

In [71]:
# Test on Karat7
beta1 = 0.2
alpha1 = 0.1

state_karat7 = reset_karat7(2)

# Run the gif cell in the annex
mygif = makeAGifSIS(karat7, state_karat7, alpha1, beta1, "part-1/Question_3_SIS_karat7", 10, 3)

┌ Info: Saved animation to 
│   fn = C:\Users\Nitro\Desktop\cours\2A\epidemic-spreading\part-1\Question_3_SIS_karat7.gif
└ @ Plots C:\Users\Nitro\.julia\packages\Plots\8GUYs\src\animation.jl:102


![out](part-1/Question_3_SIS_karat7.gif)

In [72]:
# Test on N7_2A

beta1 = 0.2
alpha1 = 0.1

state_n7_2A = reset_n7_2A(2)

mygif = makeAGifSIS(n7_2A, state_n7_2A, alpha1, beta1, "part-1/Question_3_SIS_n7_2A", 10, 3)

┌ Info: Saved animation to 
│   fn = C:\Users\Nitro\Desktop\cours\2A\epidemic-spreading\part-1\Question_3_SIS_n7_2A.gif
└ @ Plots C:\Users\Nitro\.julia\packages\Plots\8GUYs\src\animation.jl:102


![out](part-1/Question_3_SIS_n7_2A.gif)

In [73]:
# Test on Toulouse_neigh

beta1 = 0.2
alpha1 = 0.1

state_toulouse_neigh = reset_toulouse_neigh()

mygif = makeAGifSIS(toulouse_neigh, state_toulouse_neigh, alpha1, beta1, "part-1/Question_3_SIS_toulouse_neight", 20, 7)

┌ Info: Saved animation to 
│   fn = C:\Users\Nitro\Desktop\cours\2A\epidemic-spreading\part-1\Question_3_SIS_toulouse_neight.gif
└ @ Plots C:\Users\Nitro\.julia\packages\Plots\8GUYs\src\animation.jl:102


![out](part-1/Question_3_SIS_toulouse_neight.gif)

<div style="width:75%;margin:0 auto;">

### 1.4 Simulate and understand the epidemic
    
<p style="text-align:justify;">In the SIS model of this project, every disease is characterized by:
    
* The infection rate $\beta$ representing the chance of infection when being in contact with an infected individual.
* The curing rate $\alpha$ representing the chance of being cured of the disease.
* The effective spreading rate $\tau=\frac{\beta}{\alpha}$ representing the capacity of the disease to spread. More the disease infect easily ($\beta$ high) and less it is cured easily ($\alpha$ low) more $\tau$ can be high.

We are now willing to understand what are the influences of these parameters as well as the contact network shape on an epidemic.</p>

<div style="width:75%;margin:0 auto;">

<strong style="color:cadetblue">Question 4 (written):</strong> The <code>function SIS</code> you implemented launches one run of an SIS model on a given contact network. As it makes use of randomness, one run of spreading is stochastic. Then what simple estimator/method can you propose to provide a prediction of the disease spreading on a given contact network ?
   
<br>
    
<div style="background-color:#E7F1D1"> <Strong>Answer:</Strong>
            Let $nb\_infected_n$ and $nb\_susceptible_n$ be the number of people that are respectively susceptible and infected at the n-th step. <br/>
        To simplify the problem we will only work with proportions, let $i_n$ and $s_n$ the proportion of infected persons and susceptible persones, and $NV$ the number of vertices on the graph. $ i_n + s_n = 1$ <br/>
         Using SIS, we got that $ i_{n+1} = i_{n} + \frac{nb\_new\_cases_n}{NV} - \frac{nb\_cured_n}{NV} $ <br/>
    $nb\_cured_n$ is always equal to $nb\_infected_n * \alpha$ but we found two ways to estimate $nb\_new\_cases_n$<br/>
    * <Strong> from the infected persons point of view </Strong> <br/>
    Each infected person has a $\beta$ probability of infected each susceptible neighbours, which is on average $nb\_neighbours\_avg * s_n$ <br/>
    It follows that $nb\_new\_cases_n = nb\_infected_n * nb\_neighbours\_avg * s_n * \beta$ <br/>
    $$ = nb\_infected_n * (NV-  nb\_infected_n) * nb\_neighbours\_avg * \beta$$
    $$ i_{n+1} = i_n + i_n*((1-i_n)*nb\_neighbours\_avg*\beta) - i_n*\alpha$$
    $$ \fbox{$ i_{n+1} = (1 - \alpha + nb\_neighbours\_avg*\beta)*i_n - nb\_neighbours\_avg * \beta * i_n^2  $}$$<br/><br/>
    * <Strong> from the susceptible persons point of view </Strong> <br/>
    Each susceptible person is infected if <Strong>at least one</Strong> infected neighbour infect them. The probability to get infected is $ 1 -$ the probability to never get infected, which is $ (1-\beta)^{nb\_neighbours\_avg*i_n} $ so we can write $\beta_n' = 1 - (1-\beta)^{nb\_neighbours\_avg*i_n} $ the effective transmission rate in the network at the time $n$<br/>
    It follows that $ nb\_new\_cases_n =nb\_susceptible_n * \beta_n' $<br/>
    $$ i_{n+1} =i_n + (1 - i_n)*\beta_n' - \alpha i_n$$
    $$ \fbox{$ i_{n+1} = (1 - \alpha - \beta_n')i_n + \beta_n' $} $$

Since we have two estimators, I will compare them with the simulation in the annex. <br/>
We can then see that they are both pretty close to each other and quite accurate.
    </div>
    <br/>
    <div style="background-color:##00CED1"> <strong>Remark:</strong>
    We can use the first estimator to find an estimation of the limit as $n$ tends to infinity :  if the sequence converge to $l$ the equation becomes $ l = (1 - \alpha + nb\_neighbours\_avg*\beta)*l - nb\_neighbours\_avg * \beta * l^2  $
        $$ 0 = ( nb\_neighbours\_avg*\beta - \alpha) * l -  nb\_neighbours\_avg * \beta * l^2   $$
        if $l \ne 0$
        $$ 0 = ( nb\_neighbours\_avg*\beta - \alpha) -  nb\_neighbours\_avg * \beta * l   $$
        $$ \frac{nb\_neighbours\_avg*\beta - \alpha}{nb\_neighbours\_avg * \beta} =  l   $$
        $$ l = 1 - \frac{\alpha}{nb\_neighbours\_avg * \beta}$$
        with the constraint on $l$ we then know that <br/>
        $$\fbox{$\left\lbrace
        \begin{array}{cc}
        l = 0 \mbox{ if } \alpha >= nb\_neighbours\_avg * \beta \\
        l = 1 - \frac{\alpha}{nb\_neighbours\_avg * \beta} \mbox{else}
\end{array}\right.$}$$
        After verifying we can say that those estimations of the limits are somewhat accurate but not perfect at all.
</div>

<div style="width:75%;margin:0 auto;">
    
<strong style="color:cornflowerblue">Question 5 (code):</strong> Implement the <code>function Simulation_SIS</code> (respect the header and the specifications).
    
<span style="font-size:0.9em">The corrector should be able to write <code>predictions, taus = Simulation_SIS(net,nbinf,betas,alphas,t,nbsimu)</code> with your code.</span>
    
    

In [74]:
function Simulation_SIS(net,nbinf,betas,alphas,t,nbsimu)
    """Take a contact network, different diseases (defined by 
    different parameters alpha and beta), a number of initial
    infected people and process nbsimu simulations of SIS over
    t time steps. You will provide the prediction of the 
    percentage of infected at each time t as well as the 
    spreading rate of each disease.
    
    PARAMS
       net (LightGraph): graph representing the contact network
       nbinf (Int32): number of infected at the start of each 
            simulation
       betas (Array{Float64,1}): array of infection rate on edges
       alphas (Array{Float64,1}): array of curing rate on vertices
       t (Int32): number of time step
       nbsimu (Int32): number of simulations
    
    RETURNS
        (Array{Float64,2}): the prediction of the percentage of 
            infected at each time step and for each disease. The 
            first dimension contains the time steps and the second
            contains the diseases
        (Array{Float64,1}): effective spreading rate for each 
            disease
    """
    NV = nv(net)
    NE = ne(net)
    
    nb_diseases = size(alphas)[1]
    taus = zeros(Float32, nb_diseases)
    predictions = zeros(Float32, t, nb_diseases)
    
    for disease in 1:nb_diseases # For each disease
        taus[disease] = betas[disease]/alphas[disease] # Calculating tau
        for simu in 1:nbsimu # For each simulation
            state = zeros(Int32, NV)
            for _ in 1:nbinf
                randomId= Int(rand(1:NV)) # Init the infection
                while state[randomId] == 1
                    randomId= Int(rand(1:NV))
                end
                state[randomId] = 1
            end # Run the simulation
            for step in 1:t
                state = SIS(net,state,betas[disease],alphas[disease],1)
                predictions[step, disease] = predictions[step, disease] + 100*(sum(state[s] for s in 1:NV)/NV)/nbsimu
            end
        end
    end
    return predictions, taus
end

Simulation_SIS (generic function with 1 method)

<div style="width:75%;margin:0 auto;">

<strong style="color:cadetblue">Question 6 (written)</strong>: Run the 2 scripts below and describe what you see. Conclude on the influence of $\tau$, $\beta$, and $\alpha$ on an epidemic we can model with SIS.
    
<br>
    
<div style="background-color:#E7F1D1"> <strong>Answer:</strong>
    We can see that only $\tau$ as an influence on the epidemic model when times tends to infinity. And $\alpha$ and $\beta$ only influence the beginning. It's logical when you see it like this : In this model, multiplying $\alpha$ and $\beta$ is the same as multyipling the time step by the same amount.
    </div>
    


In [97]:
#######################################
### Long runtime for this cell ~30s ###
#######################################

# Script launching predictions on different diseases on karat7 and printing 
# the precentage of infected at each time step.
betas=[0.05,0.1,0.01,0.4,0.04,0.05,0.005]
alphas=[0.05,0.1,0.01,0.1,0.01,0.1,0.01]

predictions, taus = Simulation_SIS(karat7,2,betas,alphas,1000,1000)


p = Plots.plot(predictions, label=taus',xlabel="Time",ylabel="% of infected", ylim = (0,100))
savefig(p,"part-1/Question_6_simulation_karat7")


![out](part-1/Question_6_simulation_karat7.png)

In [76]:
#########################################
### Long runtime for this cell 1min30 ###
#########################################

# Same as before but applied on toulouse_neigh. May be a bit long to run.
betas = [0.05,0.1,0.15,0.25,0.35,0.45,0.55,0.65,0.75,0.85,0.95]
alphas = [0.2,0.2,0.2,0.2,0.2,0.2,0.2,0.2,0.2,0.2,0.2]

predictions, taus = Simulation_SIS(toulouse_neigh,100,betas,alphas,1000,100)

p = Plots.plot(predictions, label=taus',xlabel="Time",ylabel="% of infected")
savefig(p,"part-1/Question_6_simulation_toulouse_neight")


![out](part-1/Question_6_simulation_toulouse_neight.png)

<div style="width:75%;margin:0 auto;">

<strong style="color:cadetblue">Question 7 (written):</strong> Change the initial number of infected in the scripts above, is it in accordance with your answer in Question 2 ?
    
<br>
    
<div style="background-color:#E7F1D1"> <strong>Answer:</strong>
    Indeed, $nbinf$ has nearly no influence over time on this model. <br/>
    Although having a $nbinf$ too close to $0$ makes it possible that the disease disapear before reaching its limit. That's why on the following plot, for $nbinf = 2$ the average number of case has a limit under the normal limit, since in some simulation the virus disapear soon after the beginning. If we take it to an extreme, it's obvious that if $nbinf$ = 0, the curve will be constant. </div>

In [77]:
##########################################
### Long runtime for this cell ~6min30 ###
##########################################

betas=[0.05,0.1,0.01,0.4,0.04,0.05,0.005]
alphas=[0.05,0.1,0.01,0.1,0.01,0.1,0.01]

predictions1, taus = Simulation_SIS(karat7,2,betas,alphas,1000,1000)
predictions2, taus = Simulation_SIS(karat7,10,betas,alphas,1000,1000)
predictions3, taus = Simulation_SIS(karat7,20,betas,alphas,1000,1000)



betas = [0.05,0.1,0.15,0.25,0.35,0.45,0.55,0.65,0.75,0.85,0.95]
alphas = [0.2,0.2,0.2,0.2,0.2,0.2,0.2,0.2,0.2,0.2,0.2]

predictions4, taus = Simulation_SIS(toulouse_neigh,50,betas,alphas,1000,100)
predictions5, taus = Simulation_SIS(toulouse_neigh,100,betas,alphas,1000,100)
predictions6, taus = Simulation_SIS(toulouse_neigh,300,betas,alphas,1000,100)



(Float32[26.082998 27.871004 … 49.336998 52.245995; 22.855997 26.427998 … 71.13401 75.63401; … ; 0.0 14.122 … 79.654 81.20599; 0.0 14.160998 … 79.70102 80.84], Float32[0.25, 0.5, 0.75, 1.25, 1.75, 2.25, 2.75, 3.25, 3.75, 4.25, 4.75])

In [78]:
p1 = Plots.plot(predictions1,ylim = (0,100))
p2 = Plots.plot(predictions2,ylim = (0,100))
p3 = Plots.plot(predictions3,ylim = (0,100))

p4 = Plots.plot(predictions4,ylim = (0,100))
p5 = Plots.plot(predictions5,ylim = (0,100))
p6 = Plots.plot(predictions6,ylim = (0,100))

p = Plots.plot(p1,p2,p3,p4,p5,p6,
                layout = (2, 3), legend = false, 
                size = (900, 700),
                title = ["karat7 nbinf = 2" "karat7 nbinf = 10" "karat7 nbinf = 20" "neight nbinf = 50" "neight nbinf = 100" "neight nbinf = 300"])
savefig(p,"part-1/Question_7_influence_of_nbinf.png")

![out](part-1/Question_7_influence_of_nbinf.png)

<div style="width:75%;margin:0 auto;">

<strong style="color:cornflowerblue">Question 8 (code):</strong> Implement a script plotting the maximum percentage of infected people according to $\tau$ over 300 time steps for 3 contact networks:

* A regular graph of 200 vertices with degree 2.
* A regular graph of 200 vertices with degree 5.
* A regular graph of 200 vertices with degree 10.

You can use the function <code>random_regular_graph(n,d)</code> of LighGraphs. As you probably need to use a certain number of different values of $\tau$ to visualize something interesting (the more there are the more the figure will be smooth) you should fix $\alpha$ and make $\beta$ vary. 

<span style="font-size:0.9em">A regular graph is a graph where each vertex has the same degree.</span>

In [113]:
########################################
### Long runtime for this cell ~2min ###
########################################

# Plots of the maximum percentage of infected people according to tau over 300 time 
# steps for 3 contact networks.

a = 0.2
betas = zeros(100)
taus = zeros(100)
for i in 1:100
    betas[i] = i/400  
    taus[i] = betas[i]/a
end


g1 = random_regular_graph(200,2)
g2 = random_regular_graph(200,5)
g3 = random_regular_graph(200,10)
max1 = zeros(2,100)
max2 = zeros(2,100)
max3 = zeros(2,100)
nb_simu = 100
for i in 1:100
    simu1, ~ = Simulation_SIS(g1,10,[betas[i]],[a],300,nb_simu)
    simu2, ~ = Simulation_SIS(g2,10,[betas[i]],[a],300,nb_simu)
    simu3, ~ = Simulation_SIS(g3,10,[betas[i]],[a],300,nb_simu)
    simu1_avg = zeros(300)
    simu2_avg = zeros(300)
    simu3_avg = zeros(300)
    for step in 1:300
        simu1_avg[step] += simu1[step][1]
        simu2_avg[step] += simu2[step][1]
        simu3_avg[step] += simu3[step][1]
    end
    max1[1,i] = maximum(simu1_avg)
    max2[1,i] = maximum(simu2_avg)
    max3[1,i] = maximum(simu3_avg)
    
    tau = taus[i]

    max1[2,i] = tau
    max2[2,i] = tau
    max3[2,i] = tau
end


In [114]:
p1 = Plots.plot(max1[2,:],max1[1,:], ylim = (0,100))
p2 = Plots.plot(max2[2,:],max2[1,:], ylim = (0,100))
p3 = Plots.plot(max3[2,:],max3[1,:], ylim = (0,100))

p = Plots.plot(p1,p2,p3,
    layout = (1, 3), legend = false, 
    size = (900, 300),
    title = ["degree 2" "degree 5" "degree 10"])

savefig(p,"part-1/Question_8_influence_of_nb_neighbours.png")

![out](part-1/Question_8_influence_of_nb_neighbours.png)

<div style="width:75%;margin:0 auto;">

<strong style="color:cadetblue">Question 9 (written):</strong> Describe the figure and draw conclusions on the epidemic behavior for different degrees $d$ on regular graphs. Thus, in addition of the inner properties of the disease ($\alpha$, $\beta$, $\tau$) what other parameter is essential in the spreading ? Finally, what analogy can be done with real life from this experiment ?
    
<br>
    
<div style="background-color:#E7F1D1"> <strong>Answer:</strong>
    Thanks to the last plot, we can confirm that the number of neighbours as its role to play in the spread of the disease in a regular graph, but it's true for any graph that the number of edges is essential in the spreading as we found out in the estimator. We can say that increasing the number of neighbours is equivalent to increasing $\beta$, we saw it with our estimators.</div>

<div style="width:75%;margin:0 auto;">

## Part 2 - SIR and SAIR model
<a id='part2'></a>
    
<p style="text-align:justify;">Unfortunately SIS model is valuable for diseases we can catch back since a cured person can get ill again. This is true for the flu, the cold, etc. However COVID-19 might create immunity for whom already got it and SIS can not take into account immune or dead persons. That is why we propose in this part to consider another model more adapted to COVID-19 called SIR. It stands for Susceptible $\rightarrow$ Infected $\rightarrow$ Recovered where the susceptible group contains those who remain susceptible to the infection, the infected group consists of those who not only have the disease but are also in the contagious period of the disease, and the recovered group contains those who were ill, got cured, are not contagious and can not get ill anymore.</p>

<br>

<p style="font-size:0.9em">M. Youssef and C. Scoglio, <cite>"An individual-based approach to SIR epidemics in contact networks"</cite>, Journal of Theoretical Biology 283 (2011)</p>

<br>
    
<p style="text-align:justify;"> One limitation of SIR is that it does not model the reaction of humans when they feel the presence of the epidemic. Indeed, if feeling threaten or surrounded by infected, an individual may change its behaviors: wear mask, wash its hands, etc. This result in a smaller infection rate. That is why in this part we will also consider a variant of SIR called SAIR which stands for Susceptible $\rightarrow$ Alert $\rightarrow$ Infected $\rightarrow$ Recovered. A susceptible individual becomes infected by the infection rate $\beta_0$, an infected individual recovers and gets immune by the curing rate $\alpha$, an individual can observe the states of its neighbors, then a susceptible individual might go to the alert state if surrounded by infected individuals with an alert rate $\kappa$ on each contact with an infected, an alert inividual becomes infected by the infection rate $\beta_1$ where $0<\beta_1<\beta_0$. In our simple SAIR model, an individual can not go back to a susceptible state when he got into the alert state.</p>
    
<br>
    
<p style="font-size:0.9em"> F. Darabi Sahneh and C. Scoglio, <cite>"Epidemic Spread in Human Networks"</cite>, 50th IEEE Conf. Decision and Contol, Orlando, Florida (2011)</p>

<div style="width:75%;margin:0 auto;">

### 2.1 SIR
    
<p style="text-align:justify;">The vector containing the disease status <code>state</code> has to change a bit since we added a new state. Hence it will be an <code>Array{Int32,1}</code> where Susceptible=0, Infected=1, and Recovered=2.</p>

<div style="width:75%;margin:0 auto;">
    
<strong style="color:cornflowerblue">Question 10 (code):</strong> Implement the <code>function SIR</code> (respect the header and the specifications). You can use <code>rand</code> to translate the probabilities. Test your algorithm on <code>karat7</code>, <code>n7_2A</code>, and <code>toulouse_neigh</code> with arbitrary $\beta$, $\alpha$, and $t$. Recovered vertices should appear in a different color (<code>colorant"purple"</code>).
    
<span style="font-size:0.9em">The corrector should be able to write <code>new_state = SIR(net,state,beta,alpha,t)</code> with your code.</span>

In [81]:
function SIR(net,state,beta,alpha,t)
    """Take a contact network at a certain state and apply t time steps
    of an SIR model.
    
    PARAMS
       net (LightGraph): graph representing the contact network
       state (Array{Int32,1}): disease status of each vertex
       beta (Float64): infection rate
       alpha (Float64): curing rate
       t (Int32): number of time step
    
    RETURNS
        (Array{Int32,1}): The new state of the contact network after t time steps.
    """
    if t == 0 
        return state
    end
    
    stateTemp = zeros(Int32, nv(net))
    
    for i in 1:nv(net) # Using a temp variable to make sure that the change is step by step and changing the state of 
                       # a node won't affect the current step
        stateTemp[i] = state[i] 
    end
    
    for i in 1:nv(net) # Infect the nodes
        if state[i] == 1 # if the node is infected
            for ngb in neighbors(net,vertices(net)[i]) # for each of their neighbours
               if rand() <= beta && (stateTemp[ngb] != 2) # infect them with a probablility of beta, but not if recovered
                    stateTemp[ngb] = 1
               end
            end
        end
    end
    
    for i in 1:nv(net) # Cure the nodes
        # I don't want to try alpha as the same time as beta because I don't want infected node
        # to be healed then infected again
        if state[i] == 1 # if the node is infected
            if rand() <= alpha
                stateTemp[i] = 2
            end
        end
    end
    return SIR(net,stateTemp,beta,alpha,t-1) # Because recursive is sexy
end

SIR (generic function with 1 method)

In [82]:
# Test on Karat7

beta1 = 0.2
alpha1 = 0.1

state_karat7 = reset_karat7()

mygif = makeAGifSIR(karat7, state_karat7, alpha1, beta1, "part-2/Question_10_SIR_karat7", 20, 7)

┌ Info: Saved animation to 
│   fn = C:\Users\Nitro\Desktop\cours\2A\epidemic-spreading\part-2\Question_10_SIR_karat7.gif
└ @ Plots C:\Users\Nitro\.julia\packages\Plots\8GUYs\src\animation.jl:102


![out](part-2/Question_10_SIR_karat7.gif)

In [83]:
# Test on N7_2A

beta1 = 0.2
alpha1 = 0.1

state_n7_2A = reset_n7_2A()

mygif = makeAGifSIR(n7_2A, state_n7_2A, alpha1, beta1, "part-2/Question_10_SIR_n7_2A", 20, 7)

┌ Info: Saved animation to 
│   fn = C:\Users\Nitro\Desktop\cours\2A\epidemic-spreading\part-2\Question_10_SIR_n7_2A.gif
└ @ Plots C:\Users\Nitro\.julia\packages\Plots\8GUYs\src\animation.jl:102


![out](part-2/Question_10_SIR_n7_2A.gif)

In [84]:
# Test on Toulouse_neigh

beta1 = 0.2
alpha1 = 0.1

state_toulouse_neigh = reset_toulouse_neigh()

mygif = makeAGifSIR(toulouse_neigh, state_toulouse_neigh, alpha1, beta1, "part-2/Question_10_SIR_toulouse_neigh", 20, 7)

┌ Info: Saved animation to 
│   fn = C:\Users\Nitro\Desktop\cours\2A\epidemic-spreading\part-2\Question_10_SIR_toulouse_neigh.gif
└ @ Plots C:\Users\Nitro\.julia\packages\Plots\8GUYs\src\animation.jl:102


![out](part-2/Question_10_SIR_toulouse_neigh.gif)

<div style="width:75%;margin:0 auto;">
    
<strong style="color:cornflowerblue">Question 11 (code):</strong> Implement the <code>function Simulation_SIR</code> (respect the header and the specifications).
    
<span style="font-size:0.9em">The corrector should be able to write <code>predictions, taus = Simulation_SIR(net,nbinf,betas,alphas,t,nbsimu)</code> with your code.</span>

In [85]:
function Simulation_SIR(net,nbinf,betas,alphas,t,nbsimu)
    """Take a contact network, different diseases (defined by 
    different parameters alpha and beta), a number of initial
    infected people and process nbsimu simulations of SIR over
    t time steps. You will provide the prediction of the 
    percentage of infected at each time t as well as the 
    spreading rate of each disease.
    
    PARAMS
       net (LightGraph): graph representing the contact network
       nbinf (Int32): number of infected at the start of each 
            simulation
       betas (Array{Float64,1}): array of infection rate on edges
       alphas (Array{Float64,1}): array of curing rate on vertices
       t (Int32): number of time step
       nbsimu (Int32): number of simulations
    
    RETURNS
        (Array{Float64,3}): the prediction of the percentage of 
            infected, the percentage of susceptible and the 
            percentage of recovered at each time step and for each 
            disease. The first dimension contains the time steps,
            the second contains the diseases, and the third the status
            (Infected: [:,:,1], Recovered: [:,:,2], Susceptible: [:,:,3])
        (Array{Float64,1}): effective spreading rate for each 
            disease
    """
    NV = nv(net)
    NE = ne(net)
    
    nb_diseases = size(alphas)[1]
    taus = zeros(Float32, nb_diseases)
    predictions = zeros(Float32, t, nb_diseases,3)
    
    for disease in 1:nb_diseases
        taus[disease] = betas[disease]/alphas[disease]
        for simu in 1:nbsimu
            state = zeros(Int32, NV)
            for _ in 1:nbinf
                randomId= Int(rand(1:NV))
                while state[randomId] == 1
                    randomId= Int(rand(1:NV))
                end
                state[randomId] = 1
            end
            for step in 1:t
                state = SIR(net,state,betas[disease],alphas[disease],1)
                predictions[step, disease ,1] = predictions[step, disease,1] + 100*(sum((state[s]==1) for s in 1:NV)/NV)/nbsimu
                predictions[step, disease ,2] = predictions[step, disease,2] + 100*(sum((state[s]==2) for s in 1:NV)/NV)/nbsimu
                predictions[step, disease ,3] = predictions[step, disease,3] + 100*(sum((state[s]==0) for s in 1:NV)/NV)/nbsimu
            end
        end
    end
    return predictions, taus
end

Simulation_SIR (generic function with 1 method)

<div style="width:75%;margin:0 auto;">

<strong style="color:cadetblue">Question 12 (written):</strong> Run the script below and describe what you see. Why the infected curve does not behave the same as for SIS ? 
    
<br>

<div style="background-color:#E7F1D1"> <strong>Answer:</strong> Since people can only get sick once, the number of infected people can only decrease once everyone got infected.</div>

In [86]:
# Script launching prediction on one disease on n7_2A and plotting the percentage
# of infected, susceptible and recovered at each time step.
predictions, taus = Simulation_SIR(n7_2A,2,[0.3],[0.2],50,1000)

p = Plots.plot([predictions[:,:,1] predictions[:,:,2] predictions[:,:,3]],
            seriescolor = [:red :green :blue],
           label=["Infected" "Recovered" "Susceptible"],xlabel="t",ylabel="%")
savefig(p,"part-2/Question_12_SIR_simulation_n7_2A.png")

![out](part-2/Question_12_SIR_simulation_n7_2A.png)

<div style="width:75%;margin:0 auto;">

<strong style="color:cadetblue">Question 13 (written):</strong> As for Question 6 script 2 plot the evolution of the percentage of infected for many $\tau$. Describe what you see.
    
<br>
    
<div style="background-color:#E7F1D1"> <strong>Answer:</strong>
    Since recovered people cannot transmit the virus, the percentage of infected people quickly tend to 0 and the 1000 steps are way to much since the virus tends to quickly go to 0. That's why we did another plot with more appropriated window.<br/>
    As predicted, $\tau$ play the main role in the evolution of the spread of the infection.
    </div>

In [87]:
##########################################
### Long runtime for this cell ~1min30 ###
##########################################

# Equivalent experiment as for Question 6 script 2
betas = [0.05,0.1,0.15,0.25,0.35,0.45,0.55,0.65,0.75,0.85,0.95]
alphas = [0.2,0.2,0.2,0.2,0.2,0.2,0.2,0.2,0.2,0.2,0.2]

predictions, taus = Simulation_SIR(toulouse_neigh,100,betas,alphas,1000,100)

p = Plots.plot(predictions[:,:,1], label=taus',xlabel="Time",ylabel="% of infected")
savefig(p, "part-2/Question_13_simulation_SIR_regarding_tau.png")



better_predictions, taus = Simulation_SIR(toulouse_neigh,100,betas,alphas,30,100)

better_plot = Plots.plot(better_predictions[:,:,1], label=taus',xlabel="Time",ylabel="% of infected")
savefig(better_plot, "part-2/Question_13_better_plot_simulation_SIR_regarding_tau.png")

![out](part-2/Question_13_better_plot_simulation_SIR_regarding_tau.png)

<div style="width:75%;margin:0 auto;">

<strong style="color:cornflowerblue">Question 14 (code):</strong> Implement a script plotting the number of infected over 75 time steps for $\beta=0.3$ and $\alpha=0.2$ fixed and on 3 contact networks:
    
* A regular graph of 200 vertices with degree 2.
* A regular graph of 200 vertices with degree 5.
* A regular graph of 200 vertices with degree 10.
    

In [117]:
########################################
### Long runtime for this cell ~5min ###
########################################


# Plots of the number of infected people according to tau over 75 time 
# steps for 3 contact networks.

a = 0.2
betas = zeros(100)
taus = zeros(100)
for i in 1:100
    betas[i] = i/400  
    taus[i] = betas[i]/a
end


g1 = random_regular_graph(200,2)
g2 = random_regular_graph(200,5)
g3 = random_regular_graph(200,10)
max1 = zeros(2,100)
max2 = zeros(2,100)
max3 = zeros(2,100)
nb_simu = 100
for i in 1:100
    simu1, ~ = Simulation_SIR(g1,10,[betas[i]],[a],300,nb_simu)
    simu2, ~ = Simulation_SIR(g2,10,[betas[i]],[a],300,nb_simu)
    simu3, ~ = Simulation_SIR(g3,10,[betas[i]],[a],300,nb_simu)
    simu1_avg = zeros(300)
    simu2_avg = zeros(300)
    simu3_avg = zeros(300)
    for step in 1:300
        simu1_avg[step] += simu1[step][1][1]
        simu2_avg[step] += simu2[step][1][1]
        simu3_avg[step] += simu3[step][1][1]
    end
    max1[1,i] = maximum(simu1_avg)
    max2[1,i] = maximum(simu2_avg)
    max3[1,i] = maximum(simu3_avg)
    
    tau = taus[i]

    max1[2,i] = tau
    max2[2,i] = tau
    max3[2,i] = tau
end


In [118]:
p1 = Plots.plot(max1[2,:],max1[1,:], ylim = (0,100))
p2 = Plots.plot(max2[2,:],max2[1,:], ylim = (0,100))
p3 = Plots.plot(max3[2,:],max3[1,:], ylim = (0,100))

p = Plots.plot(p1,p2,p3,
    layout = (1, 3), legend = false, 
    size = (900, 300),
    title = ["degree 2" "degree 5" "degree 10"])

savefig(p,"part-2/Question_14_influence_of_nb_neighbours.png")

![out](part-2/Question_14_influence_of_nb_neighbours.png)

<div style="width:75%;margin:0 auto;">

<strong style="color:cadetblue">Question 15 (written):</strong> From the previous figure, explain why lockdown can be interesting when hospital places are lacking ?

<br>
    
<div style="background-color:#E7F1D1"> <strong>Answer:</strong>
    It is clear that reducing the number of contact each individual has greatly limit the spread of the virus.
    </div>

<div style="width:75%;margin:0 auto;">

### 2.2 SAIR
    
<p style="text-align:justify;">The vector containing the disease status <code>state</code> has to change a bit since we added a new state. Hence it will be an <code>Array{Int32,1}</code> where Susceptible=0, Infected=1, Recovered=2, and Alert=3.</p>

<div style="width:75%;margin:0 auto;">
    
<strong style="color:cornflowerblue">Question 16 (code):</strong> Implement the <code>function SAIR</code> (respect the header and the specifications). You can use <code>rand</code> to translate the probabilities. Test your algorithm on <code>karat7</code>, <code>n7_2A</code>, and <code>toulouse_neigh</code> with arbitrary $\beta$, $\alpha$, and $t$. Alerted vertices should appear in a different color (<code>colorant"lightgreen"</code>).
    
<span style="font-size:0.9em">The corrector should be able to write <code>new_state = SAIR(net,state,beta0,beta1,alpha,kappa,t)</code> with your code.</span>

In [90]:
function SAIR(net,state,beta0,beta1,alpha,kappa,t)
    """Take a contact network at a certain state and apply t time steps
    of an SAIR model.
    
    PARAMS
       net (LightGraph): graph representing the contact network
       state (Array{Int32,1}): disease status of each vertex
       beta0 (Float64): infection rate when not alert
       beta1 (Float64): infection rate when alert
       alpha (Float64): curing rate
       kappa (Float64): alerting rate
       t (Int32): number of time step
    
    RETURNS
        (Array{Int32,1}): The new state of the contact network after t time steps.
    """      
    if t == 0 
        return state
    end
    
    stateTemp = zeros(Int32, nv(net))
    
    for i in 1:nv(net) # Using a temp variable to make sure that the change is step by step and changing the state of 
                       # a node won't affect the current step
        stateTemp[i] = state[i] 
    end
    
    for i in 1:nv(net) # Infect the nodes
        if state[i] == 1 # if the node is infected
            for ngb in neighbors(net,vertices(net)[i]) # for each of their neighbours
                if (rand() <= beta0)  && (state[ngb] == 0) # if they're susceptible, the have beta0 chance to be infected
                    stateTemp[ngb] = 1
                end
                if (rand() <= beta1)  && (state[ngb] == 3) # if they're alerted, they have beta1 chance to be infected
                    stateTemp[ngb] = 1
                end
            end
        end
        asInfectedNgb = false #check if the node has infected neighbours
        for ngb in neighbors(net,vertices(net)[i])
            if state[ngb] == 1
                asInfectedNgb = true
            end
        end
        if (rand() <= kappa) && (state[i] == 0) && asInfectedNgb # if has an infected neighbour and is susceptible, they have kappa chance to be alerted
            stateTemp[i] = 3
        end
    end
    
    for i in 1:nv(net) # Cure the nodes
        if state[i] == 1 # if the node is infected
            if rand() <= alpha
                stateTemp[i] = 2
            end
        end
    end
    return SAIR(net,stateTemp,beta0,beta1,alpha,kappa,t-1) # Because recursive is sexy
    
end

SAIR (generic function with 1 method)

In [91]:
# Test on Karat7

beta0 = 0.2
beta1 = 0.2
alpha1 = 0.1
kappa = 0.4

state_karat7 = reset_karat7()

mygif = makeAGifSAIR(karat7, state_karat7, beta0, beta1,alpha1, kappa, "part-2/Question_16_SAIR_karat7", 10, 5)


┌ Info: Saved animation to 
│   fn = C:\Users\Nitro\Desktop\cours\2A\epidemic-spreading\part-2\Question_16_SAIR_karat7.gif
└ @ Plots C:\Users\Nitro\.julia\packages\Plots\8GUYs\src\animation.jl:102


![out](part-2/Question_16_SAIR_karat7.gif)

In [119]:
# Test on N7_2A

beta0 = 0.2
beta1 = 0.1
alpha1 = 0.1
kappa = 0.4

state_n7_2A = reset_n7_2A()

mygif = makeAGifSAIR(n7_2A, state_n7_2A, beta0, beta1,alpha1, kappa, "part-2/Question_16_SAIR_N7_2A", 20, 5)
mygif;

┌ Info: Saved animation to 
│   fn = C:\Users\Nitro\Desktop\cours\2A\epidemic-spreading\part-2\Question_16_SAIR_N7_2A.gif
└ @ Plots C:\Users\Nitro\.julia\packages\Plots\8GUYs\src\animation.jl:102


![out](part-2/Question_16_SAIR_N7_2A.gif)

In [120]:
# Test on Toulouse_neigh

beta0 = 0.2
beta1 = 0.1
alpha1 = 0.1
kappa= 0.4

state_toulouse_neigh = reset_toulouse_neigh()

mygif = makeAGifSAIR(toulouse_neigh, state_toulouse_neigh, beta0, beta1,alpha1,kappa, "part-2/Question_16_SAIR_Toulouse_neigh", 20, 5)
mygif;

┌ Info: Saved animation to 
│   fn = C:\Users\Nitro\Desktop\cours\2A\epidemic-spreading\part-2\Question_16_SAIR_Toulouse_neigh.gif
└ @ Plots C:\Users\Nitro\.julia\packages\Plots\8GUYs\src\animation.jl:102


![out](part-2/Question_16_SAIR_Toulouse_neigh.gif)

<div style="width:75%;margin:0 auto;">
    
<strong style="color:cornflowerblue">Question 17 (code):</strong> Implement the <code>function Simulation_SAIR</code> (respect the header and the specifications).
    
<span style="font-size:0.9em">The corrector should be able to write <code>predictions, taus = Simulation_SAIR(net,nbinf,betas0,betas1,alphas,kappas,t,nbsimu)</code> with your code.</span>

In [94]:
function Simulation_SAIR(net,nbinf,betas0,betas1,alphas,kappas,t,nbsimu)
    """Take a contact network, different diseases (defined by 
    different parameters alpha and beta), a number of initial
    infected people and process nbsimu simulations of SAIR over
    t time steps. You will provide the prediction of the 
    percentage of infected at each time t as well as the 
    spreading rate of each disease.
    
    PARAMS
       net (LightGraph): graph representing the contact network
       nbinf (Int32): number of infected at the start of each 
            simulation
       betas0 (Array{Float64,1}): array of infection rate when not alert on edges
       betas1 (Array{Float64,1}): array of infection rate when alert on edges
       alphas (Array{Float64,1}): array of curing rate on vertices
       kappas (Array{Float64,1}): array of alerting rate on edges
       t (Int32): number of time step
       nbsimu (Int32): number of simulations
    
    RETURNS
        (Array{Float64,3}): the prediction of the percentage of 
            infected, the percentage of susceptible and the 
            percentage of recovered at each time step and for each 
            disease. The first dimension contains the time steps,
            the second contains the diseases, and the third the status
            (Infected: [:,:,1], Recovered: [:,:,2], Susceptible: [:,:,3])
        (Array{Float64,1}): effective spreading rate for each 
            disease
    """
    NV = nv(net)
    NE = ne(net)
    
    nb_diseases = size(alphas)[1]
    taus = zeros(Float32, nb_diseases)
    predictions = zeros(Float32, t, nb_diseases,4)
    
    for disease in 1:nb_diseases
        taus[disease] = betas1[disease]/alphas[disease]
        for simu in 1:nbsimu
            state = zeros(Int32, NV)
            for _ in 1:nbinf
                randomId= Int(rand(1:NV))
                while state[randomId] == 1
                    randomId= Int(rand(1:NV))
                end
                state[randomId] = 1
            end
            for step in 1:t
                state = SAIR(net,state,betas0[disease],betas1[disease],alphas[disease],kappas[disease],1)
                predictions[step, disease ,1] = predictions[step, disease,1] + 100*(sum((state[s]==1) for s in 1:NV)/NV)/nbsimu
                predictions[step, disease ,2] = predictions[step, disease,2] + 100*(sum((state[s]==2) for s in 1:NV)/NV)/nbsimu
                predictions[step, disease ,3] = predictions[step, disease,3] + 100*(sum((state[s]==0) for s in 1:NV)/NV)/nbsimu + 100*(sum((state[s]==3) for s in 1:NV)/NV)/nbsimu
            end
        end
    end
    return predictions, taus
end

Simulation_SAIR (generic function with 1 method)

<div style="width:75%;margin:0 auto;">

<strong style="color:cadetblue">Question 18 (written):</strong> Run the script below comparing the number of infected of SAIR and SIR and comment what you see.
    
<br>
    
<div style="background-color:#E7F1D1"> <strong>Answer:</strong>
    The SAIR model have much less cases at the same time than the SIR model. We also can verify than if beta1 = beta0, the two models are roughly the same. The SAIR model highlights the importance in alerting the population and prevening measures so each individual can limit the spread of the infection.
    </div>

In [95]:
# Script launching prediction on one disease on toulouse_neigh and plotting the percentage
# of infected at each time step for SIR and SAIR.
predictions1, taus1 = Simulation_SAIR(toulouse_neigh,2,[0.2],[0.1],[0.1],[0.4],100,100)
predictions2, taus2 = Simulation_SIR(toulouse_neigh,2,[0.2],[0.1],100,100)

p = Plots.plot([predictions1[:,:,1] predictions2[:,:,1]],
           label=["SAIR" "SIR"],xlabel="t",ylabel="%")

savefig(p, "part-2/Question_18_comparing_SAIR_SIR.png")

![out](part-2/Question_18_comparing_SAIR_SIR.png)

<div style="width:75%;margin:0 auto;">

<strong style="color:cadetblue">Question 19 (written):</strong> Of course the presented SIS, SIR, and SAIR models are limitated in their modelization of the reality. Formulate few of these limitations (at least 2). 
Propose few algorithm addons/ideas (at least 2) which would make the models more complex and more accurate in regards to the reality.
    
<br>
    
<div style="background-color:#E7F1D1"> <strong>Answer:</strong><br>
    * One limitation would be that people are moving, sometimes they change neighbours even during the day by doing groceries or going to work / school. Simulating that would require a simulation that takes place on a plane and not on a graph.<br>
    * Another one would be simulating borders between contries, we could change the state array to give it two rows and the second one representing the contry, giving another $\beta$ for each border
    </div>

<div style="width:75%;margin:0 auto;">

## Part 3 - Discover patient zero
    
<p style="text-align:justify;"> In the two previous parts you may have realised that understanding and controlling the spread of epidemics on contact networks is an important task. However, information about
the origin of the epidemic could be also extremely useful to reduce or prevent future outbreaks. Thus, in this part we will focus on algorithm solutions to answer this issue.</p>
    
<p style="text-align:justify;"> The stochastic nature of infection propagation makes the estimation of the epidemic origin intrinsically hard: indeed, different initial conditions can lead to the same configuration at the observation time. Methods such as the distance centrality or the Jordan center try to approximate it. They both rely on spatial information by stating that the first infected is probably at the center of the cluster of infection. Mathematically:
    
* The jordan center is expressed as $\min_{v\in \mathcal{I}}\max_{n\in \mathcal{I}}d(v,n)$ where $\mathcal{I}$ is a connected component of the original contact network containing all infected and recovered vertices, and where $d(\cdot,\cdot)$ is the distance (= the shortest path) between 2 vertices (if not weighted graph each edge accounts for 1 unit). 
* The distance centrality is expressed as $\min_{v\in \mathcal{I}}\sum_{n\in \mathcal{I}}d(v,n)(\delta_{n,I} + \delta_{n,R}/\alpha)$, where $\delta_{n,I}=1$ if the vertex n is infected ($=0$ otherwise), and where $\delta_{n,R}=1$ if the vertex n is recovered ($=0$ otherwise). You may note that in distance centrality we increase the weight of the recovered vertices by a factor $1/\alpha$, it translates the fact that recovered vertices tend to be closer to the origin of the epidemic since they probably got ill before.
    
    
We formulate the problem as follow: given a contact network and a snapshot of epidemic spread at a certain time, determine the infection source. A snapshot is a given <code>state</code> array for a contact network.</p>

<br>
    
<p style="font-size:0.9em"> A. Y. Lokhov, M. Mézard, H. Ohta, and L. Zdeborová, <cite>"Inferring the origin of an epidemic with a dynamic message-passing algorithm"</cite>, Physical Review (2014)</p>

<div style="width:75%;margin:0 auto;">
    
<strong style="color:cornflowerblue">Question 20 (code):</strong> Implement the <code>function jordan</code> (respect the header and the specifications). You will need to use the function <code>dijkstra_shortest_paths</code> of the LighGraphs library, refer to the doc for more information. If there are multiple minimal vertices, then return the first one.
    
<span style="font-size:0.9em">The corrector should be able to write <code>zero = jordan(g,state,alpha)</code> with your code.</span>

In [122]:
function jordan(g,state)
    """Find patient zero by mean of the jordan center method.
    
    PARAMS
        g (LightGraph): graph representing the contact network
        state (Array{Int32,1}): disease status of each vertex
    
    RETURNS
        (Int32): the patient zero vertex number 
    """
    L = g
    min = +Inf
    zero = 0
    for v in 1:nv(L)
        if state[v] != 0
            ds = dijkstra_shortest_paths(L, v)
            max = -Inf
            for n in 1:nv(L)
                if state[n] != 0 && ds.dists[n] > max 
                    max = ds.dists[n]
                end
            end
            if min > max 
                min = max
                zero = v
            end
        end
    end
    return zero
end

jordan (generic function with 1 method)

<div style="width:75%;margin:0 auto;">
    
<strong style="color:cornflowerblue">Question 21 (code):</strong> Implement the <code>function distance</code> (respect the header and the specifications). You will need to use the function <code>dijkstra_shortest_paths</code> of the LighGraphs library, refer to the doc for more information. If there are multiple minimal vertices, then return the first one.
    
<span style="font-size:0.9em">The corrector should be able to write <code>zero = distance(g,state,alpha)</code> with your code.</span>

In [123]:
function distance(g,state,alpha=1)
    """Find patient zero by mean of the distance centrality method.
    
    PARAMS
        g (LightGraph): graph representing the contact network
        state (Array{Int32,1}): disease status of each vertex
        alpha (Float64): curing rate
    
    RETURNS
        (Int32): the patient zero vertex number 
    """
    L = g
    zero = 0
    min = +Inf
    
    deltaI = zeros(nv(L))
    deltaR = zeros(nv(L))
    for i in 1:nv(L)
        if state[i] == 2
            deltaR[i] = 1/alpha
        end
        if state[i] == 1
            deltaI[i] = 1
        end
    end
    
    for v in 1:nv(L)
        sum = 0
        ds = dijkstra_shortest_paths(L, v)
        for n in 1:nv(L)
            sum = sum + ds.dists[n]*(deltaI[n] + deltaR[n])
        end
        
        if sum < min
            min = sum
            zero = v
        end
    end
        
    return zero 
end

distance (generic function with 2 methods)

<div style="width:75%;margin:0 auto;">

<strong style="color:cadetblue">Question 22 (written):</strong> Run the 3 following scripts using your functions <code>jordan</code> and <code>distance</code> and comment on the results.
    
The contact network is karat7 for 2 different patient zero and a $50\times 50$ grid. The real patient zero ("Z"), your jordan ("J") and distance ("C") approximations are appearing in <code>colorant"lightblue"</code>.
    
<br>
    
<div style="background-color:#E7F1D1"> <strong> Answer:</strong>
    We can see that depending the simulation the algorithms are quite effective but not always. It's because they tend to priorize nodes with a lot of neighbours. In any case, if they fail to find the real patient zero, they may have found one of their neighbours. </div>

In [124]:
# Loading a snapshot of karat7
@load "karat7_Q22_1.jld2" g state pat_zero alpha beta loc_x loc_y

# Run the patient zero finding function
cent_pat_zero = distance(g,state,alpha)
jor_pat_zero = jordan(g,state)

# Some display options 
labels = Array{String, 1}(undef,nv(g))
for k=1:nv(g)
    if state[k]==1
        labels[k]="I"
    elseif state[k]==2
        labels[k]="R"
    else
        labels[k]="S"
    end
end

if cent_pat_zero==jor_pat_zero==pat_zero
    labels[cent_pat_zero]="C+J+Z"
elseif cent_pat_zero==jor_pat_zero
    labels[cent_pat_zero]="C+J"
    labels[pat_zero]="Z"
elseif cent_pat_zero==pat_zero
    labels[cent_pat_zero]="C+Z"
    labels[jor_pat_zero]="J"
elseif jor_pat_zero==pat_zero
    labels[jor_pat_zero]="J+Z"
    labels[cent_pat_zero]="C"
else
    labels[cent_pat_zero]="C"
    labels[jor_pat_zero]="J"
    labels[pat_zero]="Z"
end

nodecolor = [colorant"lightseagreen", colorant"orange", colorant"purple"]
colors = nodecolor[state + ones(Int32,nv(g))]
colors[pat_zero] = colorant"lightblue"
colors[cent_pat_zero] = colorant"lightblue"
colors[jor_pat_zero] = colorant"lightblue"

# Display
draw(PNG("part-3/karat7_Q22_1.png", 20cm, 20cm), gplot(g,loc_x,loc_y,nodefillc=colors,nodelabel=labels))
gplot(g,loc_x,loc_y,nodefillc=colors,nodelabel=labels);

![out](part-3/karat7_Q22_1.png)

In [125]:

# Loading a snapshot of karat7
@load "karat7_Q22_2.jld2" g state pat_zero alpha beta loc_x loc_y

# Run the patient zero finding function
cent_pat_zero = distance(g,state,alpha)
jor_pat_zero = jordan(g,state)

# Some display options 
labels = Array{String, 1}(undef,nv(g))
for k=1:nv(g)
    if state[k]==1
        labels[k]="I"
    elseif state[k]==2
        labels[k]="R"
    else
        labels[k]="S"
    end
end

if cent_pat_zero==jor_pat_zero==pat_zero
    labels[cent_pat_zero]="C+J+Z"
elseif cent_pat_zero==jor_pat_zero
    labels[cent_pat_zero]="C+J"
    labels[pat_zero]="Z"
elseif cent_pat_zero==pat_zero
    labels[cent_pat_zero]="C+Z"
    labels[jor_pat_zero]="J"
elseif jor_pat_zero==pat_zero
    labels[jor_pat_zero]="J+Z"
    labels[cent_pat_zero]="C"
else
    labels[cent_pat_zero]="C"
    labels[jor_pat_zero]="J"
    labels[pat_zero]="Z"
end

nodecolor = [colorant"lightseagreen", colorant"orange", colorant"purple"]
colors = nodecolor[state + ones(Int32,nv(g))]
colors[pat_zero] = colorant"lightblue"
colors[cent_pat_zero] = colorant"lightblue"
colors[jor_pat_zero] = colorant"lightblue"

# Display
draw(PNG("part-3/karat7_Q22_2.png", 20cm, 20cm), gplot(g,loc_x,loc_y,nodefillc=colors,nodelabel=labels))

![out](part-3/karat7_Q22_2.png)

In [126]:
# Loading a snapshot of grid50
@load "grid50_Q22.jld2" g state pat_zero alpha beta loc_x loc_y

# Run the patient zero finding function
cent_pat_zero = distance(g,state,alpha)
jor_pat_zero = jordan(g,state)

# Some display options 
labels = Array{String, 1}(undef,nv(g))
for k=1:nv(g)
    if state[k]==1
        labels[k]="I"
    elseif state[k]==2
        labels[k]="R"
    else
        labels[k]="S"
    end
end

if cent_pat_zero==jor_pat_zero==pat_zero
    labels[cent_pat_zero]="C+J+Z"
elseif cent_pat_zero==jor_pat_zero
    labels[cent_pat_zero]="C+J"
    labels[pat_zero]="Z"
elseif cent_pat_zero==pat_zero
    labels[cent_pat_zero]="C+Z"
    labels[jor_pat_zero]="J"
elseif jor_pat_zero==pat_zero
    labels[jor_pat_zero]="J+Z"
    labels[cent_pat_zero]="C"
else
    labels[cent_pat_zero]="C"
    labels[jor_pat_zero]="J"
    labels[pat_zero]="Z"
end

nodecolor = [colorant"lightseagreen", colorant"orange", colorant"purple"]
colors = nodecolor[state + ones(Int32,nv(g))]
colors[pat_zero] = colorant"lightblue"
colors[cent_pat_zero] = colorant"lightblue"
colors[jor_pat_zero] = colorant"lightblue"

# Display
draw(PNG("part-3/grid50_Q22.png", 100cm, 100cm), gplot(g,loc_x,loc_y,nodefillc=colors,nodelabel=labels))

![out](part-3/grid50_Q22_zoom.png)

# ANNEX
## 1. Displaying the estimators


In [128]:
function estimator_SIS_infected_pov(net,nbinf,betas,alphas,t)
    """
    Estimate the proportion of infected people for a given net.
    It will use the estimator with the infected point of view.
    
    PARAMS
        net (LightGraph): graph representing the contact network
        nbfin (Float64): starting number of case
        betas (Array{Float64,1}): array of infection rates
        alphas (Array{Float64,1}): array of curing rates
        t (float64): number of steps
    
    RETURNS
        prediction(Array{Float64,2}): array such that prediction[s, d] returns 
                                    the estimated number of infected people at the step s for the disease d
        taus(Array{Float64,1}): array countaing tau for each pair of beta / alpha
    """
    NV = nv(net)
    NE = ne(net)
    
    nb_diseases = size(alphas)[1]
    taus = zeros(Float32, nb_diseases)
    predictions = zeros(Float32, t, nb_diseases)
    nb_ngb = 2*NE/NV
    for disease in 1:nb_diseases
        a = -1 * betas[disease]*nb_ngb
        b = 1 - alphas[disease] - a
        predictions[1,disease] = nbinf/NV
        taus[disease] = betas[disease]/alphas[disease]
        for step in 2:t
            predictions[step,disease] =  a * predictions[step-1,disease]^2 + b * predictions[step-1,disease]
        end
    end
    return predictions, taus
end

function estimator_SIS_susceptible_pov(net,nbinf,betas,alphas,t)
    """
    Estimate the proportion of infected people for a given net.
    It will use the estimator with the susceptible point of view.
    
    PARAMS
        net (LightGraph): graph representing the contact network
        nbfin (Float64): starting number of case
        betas (Array{Float64,1}): array of infection rates
        alphas (Array{Float64,1}): array of curing rates
        t (float64): number of steps
    
    RETURNS
        prediction(Array{Float64,2}): array such that prediction[s, d] returns 
                                    the estimated number of infected people at the step s for the disease d
        taus(Array{Float64,1}): array countaing tau for each pair of beta / alpha
    """
    NV = nv(net)
    NE = ne(net)
    
    nb_diseases = size(alphas)[1]
    taus = zeros(Float32, nb_diseases)
    predictions = zeros(Float32, t, nb_diseases)
    nb_ngb = 2*NE/NV
    for disease in 1:nb_diseases
        predictions[1,disease] = nbinf/NV
        taus[disease] = betas[disease]/alphas[disease]
        for step in 2:t
            betaprime = (1-(1-betas[disease])^(nb_ngb*predictions[step-1,disease]))
            a = 1 - alphas[disease] - betaprime
            predictions[step,disease] =  a*predictions[step-1,disease] + betaprime
        end
    end
    return predictions, taus
end



estimator_SIS_susceptible_pov (generic function with 1 method)

In [129]:
betas=[0.05,0.1,0.01,0.4,0.04,0.05,0.005]
alphas=[0.05,0.1,0.01,0.1,0.01,0.1,0.01]
simulations1, taus = Simulation_SIS(karat7,2,betas,alphas,1000,1000)

betas = [0.05,0.1,0.15,0.25,0.35,0.45,0.55,0.65,0.75,0.85,0.95]
alphas = [0.2,0.2,0.2,0.2,0.2,0.2,0.2,0.2,0.2,0.2,0.2]
simulations2, taus = Simulation_SIS(toulouse_neigh,100,betas,alphas,1000,100)


(Float32[8.968997 9.785999 … 21.040998 22.388008; 8.129001 10.129002 … 47.707005 53.102; … ; 0.0 14.442997 … 79.58301 81.088005; 0.0 14.420003 … 79.55502 81.096], Float32[0.25, 0.5, 0.75, 1.25, 1.75, 2.25, 2.75, 3.25, 3.75, 4.25, 4.75])

In [131]:
betas=[0.05,0.1,0.01,0.4,0.04,0.05,0.005]
alphas=[0.05,0.1,0.01,0.1,0.01,0.1,0.01]

predictions1, taus = estimator_SIS_infected_pov(karat7,2,betas,alphas,1000)
predictions2, taus = estimator_SIS_susceptible_pov(karat7,2,betas,alphas,1000)


pi1 = Plots.plot(predictions1,ylim = (0,1))
ps1 = Plots.plot(predictions2,ylim = (0,1))
s1 = Plots.plot(simulations1,ylim = (0,100))

betas = [0.05,0.1,0.15,0.25,0.35,0.45,0.55,0.65,0.75,0.85,0.95]
alphas = [0.2,0.2,0.2,0.2,0.2,0.2,0.2,0.2,0.2,0.2,0.2]

predictions1, taus = estimator_SIS_infected_pov(toulouse_neigh,100,betas,alphas,1000)
predictions2, taus = estimator_SIS_susceptible_pov(toulouse_neigh,100,betas,alphas,1000)

pi2 = Plots.plot(predictions1,ylim = (0,1))
ps2 = Plots.plot(predictions2,ylim = (0,1))
s2 = Plots.plot(simulations2,ylim = (0,100))




p = Plots.plot(pi1, ps1,s1, pi2, ps2,s2, 
            layout = (2, 3), legend = false, 
            size = (900, 700),
            title=["karat7 infected estimator" "karat7 susceptible estimator" "karat7 simulation" "neigh infected estimator" "neigh susceptible estimator" "neigh simulation"])

savefig(p,"Annex/comparing_estimators_simulation.png")


![out](Annex/comparing_estimators_simulation.png)

## 2 Comparing the estimators 

* By comparing the plots of the estimators with the plots of the simulation, we see that the 

In [148]:
# Setting up the simulation, the same

betas=[0.05,0.1,0.01,0.4,0.04,0.05,0.005]
alphas=[0.05,0.1,0.01,0.1,0.01,0.1,0.01]

simulation_karat7, taus = Simulation_SIS(karat7,2,betas,alphas,1000,1000)
prediction1_karat7, taus = estimator_SIS_susceptible_pov(karat7,2,betas,alphas,1000)
prediction2_karat7, taus = estimator_SIS_infected_pov(karat7,2,betas,alphas,1000)


betas = [0.05,0.1,0.15,0.25,0.35,0.45,0.55,0.65,0.75,0.85,0.95]
alphas = [0.2,0.2,0.2,0.2,0.2,0.2,0.2,0.2,0.2,0.2,0.2]

simulation_toulouse_neigh, taus = Simulation_SIS(toulouse_neigh,10,betas,alphas,1000,100)
prediction1_toulouse_neigh, taus = estimator_SIS_susceptible_pov(toulouse_neigh,10,betas,alphas,100)
prediction2_toulouse_neigh, taus = estimator_SIS_infected_pov(toulouse_neigh,10,betas,alphas,100)

(Float32[0.01 0.01 … 0.01 0.01; 0.00898901 0.00997802 … 0.02481317 0.026791189; … ; 3.2080732f-7 0.008210521 … 0.88223517 0.89463145; 2.8869448f-7 0.0081954105 … 0.88223517 0.89463145], Float32[0.25, 0.5, 0.75, 1.25, 1.75, 2.25, 2.75, 3.25, 3.75, 4.25, 4.75])

In [149]:
simulation_karat7_prop[:,:] = simulation_karat7[:,:]/100
simulation_toulouse_neigh_prop[:,:] = simulation_toulouse_neigh[:,:]/100

function accuracy(mat1, mat2)
    """ Calculs the average absolute distance and the average relative distance between two matrix
    
    PARAMS
    mat1 (Array{Float64,2}): array of numbers
    mat1 (Array{Float64,2}): array of numbers
    
    RETURNS
    distance_avg (Float64): average absolute distance
    distance_rel (Float64): average relative distance 
    """
    I = size(mat1)[1]
    J = size(mat1)[2]
    distance_avg = 0
    distance_rel = 0
    for i in 1:I
        for j in 1:J
            distance_avg += abs(mat1[i,j]-mat2[i,j])
            if mat2[i,j] != 0
                distance_rel += abs(mat1[i,j]-mat2[i,j]) / mat2[i,j]
            end
        end
    end
    return distance_avg / (I*J), distance_rel / (I*J)
end

println(" with karat7 (susceptible model vs avg, infected model vs avg, distance between both models )")
println("average distance | average relative distance")
println(accuracy(prediction1_karat7,simulation_karat7_prop))
println(accuracy(prediction2_karat7,simulation_karat7_prop))
println(accuracy(prediction1_karat7,prediction2_karat7))

println()
println(" with Toulouse neigh (susceptible model vs avg, infected model vs avg, distance between both models )")
println("average distance | average relative distance")
println(accuracy(prediction1_toulouse_neigh,simulation_toulouse_neigh_prop))
println(accuracy(prediction2_toulouse_neigh,simulation_toulouse_neigh_prop))
println(accuracy(prediction1_toulouse_neigh,prediction2_toulouse_neigh))

 with karat7 (susceptible model vs avg, infected model vs avg, distance between both models )
average distance | average relative distance
(0.09212597f0, 0.18728976f0)
(0.10844059f0, 0.21187122f0)
(0.016351955f0, 0.020720715f0)

 with Toulouse neigh (susceptible model vs avg, infected model vs avg, distance between both models )
average distance | average relative distance
(0.06028944f0, 0.26566008f0)
(0.08472727f0, 0.30885392f0)
(0.037466627f0, 0.17083272f0)


## Making gifs !

For our viewing pleasure, we made a function alloing us to diplay simulation into gifs, we made 3 functions, one for each model. You will need to run this code in order to run some cells.

In [66]:
# This whole cell is dedicated to the function makeAGif that allows me to make a pretty gif of our simulations


import Cairo
using Printf
using Colors

function makeAGifSIS(graph, state, alpha, beta, name, nbSteps, fps)
    """ Display a gif of a SIS simulation
    
    PARAMS
       graph (LightGraph): graph representing the contact network
       state (Array{Int32,1}): disease status of each vertex
       alpha (Float64): curing rate
       beta (Float64): infection rate
       name (String): name of the file
       nbSteps (Int32): number of time step
       fps (Int32): number of frames per seconds
    
    RETURNS
        gif (gif file): gif of the simulation
    """

    anim=Animation()
    NV = nv(graph)
    state_copy = state
    loc_x, loc_y = spring_layout(graph,MAXITER=200,C=1.)
        
    nodecolor = [colorant"lightseagreen", colorant"orange", colorant"purple", colorant"lightgreen"]
    nodefillc = nodecolor[state_copy + ones(Int32,NV)]
    p=gplot(graph, loc_x, loc_y, nodefillc=nodefillc)
    
    output = compose(p,(context(), Compose.text(1, 1, name)),(context(), rectangle(), fill("white")))
    j=length(anim.frames) + 1
    tmpfilename=joinpath(anim.dir,@sprintf("%06d.png",j))
    Compose.draw(PNG(tmpfilename),output)
    push!(anim.frames, tmpfilename)
    
    for i in 1:nbSteps

        state_copy = SIS(graph,state_copy,beta,alpha,1)


        nodecolor = [colorant"lightseagreen", colorant"orange"]
        nodefillc = nodecolor[state_copy + ones(Int32,NV)]
        p=gplot(graph, loc_x, loc_y, nodefillc=nodefillc)

        output = compose(p,(context(), Compose.text(1, 1, name)),(context(), rectangle(), fill("white")))
        j=length(anim.frames) + 1
        tmpfilename=joinpath(anim.dir,@sprintf("%06d.png",j))
        Compose.draw(PNG(tmpfilename),output)
        push!(anim.frames, tmpfilename)

    end

    return gif(anim, string(name,".gif"), fps = fps )
end

function makeAGifSIR(graph, state, alpha, beta, name, nbSteps, fps)
    """ Display a gif of a SIR simulation
    
    PARAMS
       graph (LightGraph): graph representing the contact network
       state (Array{Int32,1}): disease status of each vertex
       alpha (Float64): curing rate
       beta (Float64): infection rate
       name (String): name of the file
       nbSteps (Int32): number of time step
       fps (Int32): number of frames per seconds
    
    RETURNS
        gif (gif file): gif of the simulation
    """
    anim=Animation()
    NV = nv(graph)
    state_copy = state
    loc_x, loc_y = spring_layout(graph,MAXITER=200,C=1.)
    
    nodecolor = [colorant"lightseagreen", colorant"orange", colorant"purple", colorant"lightgreen"]
    nodefillc = nodecolor[state_copy + ones(Int32,NV)]
    p=gplot(graph, loc_x, loc_y, nodefillc=nodefillc)
        
    output = compose(p,(context(), Compose.text(1, 1, name)),(context(), rectangle(), fill("white")))
    j=length(anim.frames) + 1
    tmpfilename=joinpath(anim.dir,@sprintf("%06d.png",j))
    Compose.draw(PNG(tmpfilename),output)
    push!(anim.frames, tmpfilename)
    
    
    for i in 1:nbSteps

        state_copy = SIR(graph,state_copy,beta,alpha,1)


        nodecolor = [colorant"lightseagreen", colorant"orange", colorant"purple"]
        nodefillc = nodecolor[state_copy + ones(Int32,NV)]
        p=gplot(graph, loc_x, loc_y, nodefillc=nodefillc)

        output = compose(p,(context(), Compose.text(1, 1, name)),(context(), rectangle(), fill("white")))
        j=length(anim.frames) + 1
        tmpfilename=joinpath(anim.dir,@sprintf("%06d.png",j))
        Compose.draw(PNG(tmpfilename),output)
        push!(anim.frames, tmpfilename)

    end

    return gif(anim, string(name,".gif"), fps = fps )
end

function makeAGifSAIR(graph, state, beta0, beta1, alpha, kappa, name, nbSteps, fps)
    """ Display a gif of a SIAI simulation
    
    PARAMS
       graph (LightGraph): graph representing the contact network
       state (Array{Int32,1}): disease status of each vertex
       beta0 (Float64): infection rate for non alerted people
       beta1 (Float64): infection rate for alerted people
       alpha (Float64): curing rate
       kappa (Float64): alerting rate
       name (String): name of the file
       nbSteps (Int32): number of time step
       fps (Int32): number of frames per seconds
    
    RETURNS
        gif (gif file): gif of the simulation
    """
    anim=Animation()
    NV = nv(graph)
    state_copy = state
    loc_x, loc_y = spring_layout(graph,MAXITER=200,C=1.)
    
    nodecolor = [colorant"lightseagreen", colorant"orange", colorant"purple", colorant"lightgreen"]
    nodefillc = nodecolor[state_copy + ones(Int32,NV)]
    p=gplot(graph, loc_x, loc_y, nodefillc=nodefillc)
    
    output = compose(p,(context(), Compose.text(1, 1, name)),(context(), rectangle(), fill("white")))
    j=length(anim.frames) + 1
    tmpfilename=joinpath(anim.dir,@sprintf("%06d.png",j))
    Compose.draw(PNG(tmpfilename),output)
    push!(anim.frames, tmpfilename)
    
    for i in 1:nbSteps
        
        state_copy = SAIR(graph,state_copy,beta0,beta1,alpha,kappa,1)


        nodecolor = [colorant"lightseagreen", colorant"orange", colorant"purple", colorant"lightgreen"]
        nodefillc = nodecolor[state_copy + ones(Int32,NV)]
        p=gplot(graph, loc_x, loc_y, nodefillc=nodefillc)

        output = compose(p,(context(), Compose.text(1, 1, name)),(context(), rectangle(), fill("white")))
        j=length(anim.frames) + 1
        tmpfilename=joinpath(anim.dir,@sprintf("%06d.png",j))
        Compose.draw(PNG(tmpfilename),output)
        push!(anim.frames, tmpfilename)
    end
        
    return gif(anim, string(name,".gif"), fps = fps )
end

makeAGifSAIR (generic function with 1 method)